In [1]:
from param_def import chi,sigma,prop_gp,capacities_rate,prop_all_g_prefer,capacities
import numpy as np
from util import *
from itertools import compress
# Import HyperOpt Library
from hyperopt import tpe, hp, fmin
import numpy as np
from BayesEst import bayesian_est

In [2]:
grade_estimated_gr = []
for i in range(len(chi)):
    grade_estimated_gr.append(np.load(f'grade_estimated_gr{i+1}.npy'))
grade_estimated = grades_gr_to_grades_col(grade_estimated_gr)
#%%
stud_pref = []
for i in range(len(chi)):
    stud_pref.append(np.load(f'stud_pref_gr{i+1}.npy').tolist())

In [3]:
def market_clear(Pa, Pb, prop, capA, capB, prefi, prefii, sigmai, sigmaii, cori, corii):
    f1 = prop*prefi*(1 - cdf(Pa, sigmai)) + (1 - prop)*prefii*(1 - cdf(Pa, sigmaii)) + prop*(1 - prefi)*cdfmsi(Pa, Pb, sigmai, cori) + (1 -prop)*(1 - prefii)*cdfmsi(Pa, Pb, sigmaii, corii) - capA
    f2 = prop*(1 - prefi)*(1 - cdf(Pb, sigmai)) + (1 - prop)*(1 - prefii)*(1 - cdf(Pb, sigmaii)) + prop*prefi*cdfmis(Pa, Pb, sigmai, cori) + (1 -prop)*prefii*cdfmis(Pa, Pb, sigmaii, corii) - capB
    return f1, f2

In [4]:
def solv_mc(prop, capA, capB, prefi, prefii, sigmai, sigmaii, cori, corii):
    func = lambda P: market_clear(P[0], P[1], prop, capA, capB, prefi, prefii, sigmai, sigmaii, cori, corii)
    return fsolve(func, x0 = np.array([0.5, 0.5]))

In [5]:
sigma_i = np.sqrt(chi[0]**2 + sigma[0]**2)
sigma_ii = np.sqrt(chi[1]**2 + sigma[1]**2)
cor_i = chi[0]**2/sigma_i**2
cor_ii = chi[1]**2/sigma_ii**2

In [6]:
print(cor_i,cor_ii)

0.8999999999999998 0.4999999999999999


In [7]:
P_A,P_B = solv_mc(prop_gp[0],capacities_rate[0],capacities_rate[1],
                  prop_all_g_prefer[0],prop_all_g_prefer[1],sigma_i,sigma_ii,cor_i,cor_ii)


In [8]:
cutoff_values = [P_A,P_B]

In [9]:
P_A,P_B

(np.float64(0.5275604686551243), np.float64(1.9926546637487335))

In [10]:
welfare_metrics(cutoff_values,grade_estimated,stud_pref)

Proportion of students in group 0 with no offer 0.5026107461680984
Proportion of students in group 0 with only a second preference offer 0.08792319353208691
Proportion of students in group 0 with first choice offer 0.4094660602998147
Proportion of students in group 1 with no offer 0.5056141926790928
Proportion of students in group 1 with only a second preference offer 0.18246126207051425
Proportion of students in group 1 with first choice offer 0.311924545250393


In [11]:
student_by_col(cutoff_values,grade_estimated,stud_pref)

Number of student admitted to college 1 in group 0: 1508
Number of student admitted to college 2 in group 0: 1445
Number of student admitted to college 1 in group 1: 2867
Number of student admitted to college 2 in group 1: 1536


(4375, 2981)

In [12]:
print(capacities)

[4453, 2968]


In [4]:
best = {'x': 0.4993072172374298, 'y': 0.2384378586240428}
new_cutoff_values = [best['x'],best['y']]
new_cutoff_values

[0.4993072172374298, 0.2384378586240428]

In [5]:
updated_grade_estimated = bayes_update_grade(new_cutoff_values[0],new_cutoff_values[1],grade_estimated,chi,sigma,bayes_type='right')

In [6]:
welfare_metrics(new_cutoff_values,updated_grade_estimated,stud_pref)

Proportion of students in group 0 with no offer 0.49772612430520463
Proportion of students in group 0 with only a second preference offer 0.08708101734882938
Proportion of students in group 0 with first choice offer 0.415192858345966
Proportion of students in group 1 with no offer 0.5019088255108916
Proportion of students in group 1 with only a second preference offer 0.1843700875814058
Proportion of students in group 1 with first choice offer 0.3137210869077027


In [7]:
student_by_col(new_cutoff_values,updated_grade_estimated,stud_pref)

Number of student admitted to college 1 in group 0: 1494
Number of student admitted to college 2 in group 0: 1488
Number of student admitted to college 1 in group 1: 2909
Number of student admitted to college 2 in group 1: 1527


(4403, 3015)